In [1]:
import findspark

findspark.init()

from pyspark import SparkContext
import numpy as np


sc = SparkContext("local[*]", "Media practica")


24/10/19 10:17:55 WARN Utils: Your hostname, Ciruu resolves to a loopback address: 127.0.1.1; using 10.150.83.214 instead (on interface wlp2s0)
24/10/19 10:17:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/19 10:17:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
import math
rdd = sc.textFile("/home/ciruu/CLionProjects/practica2SPAI/botnet_tot_syn_l.csv")

In [3]:
def readRow(row):
    arr = row.split(",")
    lst = np.array(arr).astype(np.float64)
    return np.array([float(x) for x in lst[:-1]]), int(lst[-1])

def suma(el1, el2):
    return el1 + el2

In [4]:
rdd1 = rdd.map(readRow)
print(rdd1.take(2))

num = rdd.count()
print(num)

# Dividimos entre el número total de filas
def divide_by_num(tup):
    tup[0] = tup[0] / num
    return tup


rdd2 = rdd1.reduce(lambda x, y: (x[0] + y[0], x[1]))

print(rdd2)
medias = rdd2[0] / num
# Calculamos la media por cada columna con índice I
#medias = rdd3.collect()
print(f"MEDIAS: {medias}")
#print(medias)

[(array([9.01278427e+00, 1.67299998e+03, 2.19999885e+01, 9.99999745e-01,
       6.19998877e+01, 6.99998079e+01, 1.30000002e+01, 2.99999998e+00,
       1.99000002e+02, 2.46836957e+09, 2.46837255e+09]), 1), (array([3.59999909e+03, 4.82065758e+04, 1.33629998e+04, 1.00000192e+00,
       2.62999242e+02, 8.29998876e+01, 1.39999452e+01, 5.00000003e+00,
       2.16999996e+02, 1.53904420e+09, 2.46836839e+09]), 0)]
1000000


(array([1.28153827e+09, 2.12827672e+10, 6.94805718e+09, 6.26311961e+10,
       1.22198595e+14, 1.57222361e+13, 9.10731393e+06, 1.88975480e+06,
       1.24270159e+08, 2.13060301e+15, 2.26149180e+15]), 1)
MEDIAS: [1.28153827e+03 2.12827672e+04 6.94805718e+03 6.26311961e+04
 1.22198595e+08 1.57222361e+07 9.10731393e+00 1.88975480e+00
 1.24270159e+02 2.13060301e+09 2.26149180e+09]


In [5]:
from math import sqrt
# Función para obtener la media de un índice en concreto en un array
def get_media_from_tup_array(arr, index):
    for e in arr:
        if e[0] == index:
            return e[1]
        
        
# Función para calcular la varianza a partir de una tupla
def calc_var(tup):
    #mean = get_media_from_tup_array(medias, tup[0])
    return ((tup[0] - medias) ** 2), tup[1]

In [6]:
# Caćulo de la varianza a partir de las tuplas
rdd4 = rdd1.map(calc_var)
print(rdd4.take(2))
rdd5 = rdd4.reduce(lambda x, y: (x[0] + y[0], x[1]))



print(f"VARIANZA: {rdd5[0]/num}")

[(array([1.61932112e+06, 3.84542970e+08, 4.79702682e+07, 3.92254146e+09,
       1.49324814e+16, 2.47186507e+14, 1.51530066e+01, 1.23264437e+00,
       5.58454938e+03, 1.14086248e+17, 4.27996434e+16]), 1), (array([5.37526055e+06, 7.24891472e+08, 4.11514894e+07, 3.92254146e+09,
       1.49324323e+16, 2.47186098e+14, 2.39378410e+01, 9.67362543e+00,
       8.59882264e+03, 3.49941831e+17, 4.27979244e+16]), 0)]


VARIANZA: [2.57889305e+06 5.81652602e+08 2.68794233e+08 1.80268802e+10
 5.47189003e+16 1.94690792e+15 2.78966275e+01 4.41565337e+00
 8.16187974e+03 5.09572461e+17 1.69398542e+18]


In [7]:
#print(rdd5)
rddx = rdd1.map(lambda x: (x[0])).stdev()
print(rddx)
print(f"DESVIACIÓN ESTÁNDAR: {np.sqrt(rdd5[0]/num)}")

[1.60589322e+03 2.41174750e+04 1.63949454e+04 1.34264218e+05
 2.33920714e+08 4.41237795e+07 5.28172581e+00 2.10134561e+00
 9.03431222e+01 7.13843443e+08 1.30153195e+09]
DESVIACIÓN ESTÁNDAR: [1.60589322e+03 2.41174750e+04 1.63949454e+04 1.34264218e+05
 2.33920714e+08 4.41237795e+07 5.28172581e+00 2.10134561e+00
 9.03431222e+01 7.13843443e+08 1.30153195e+09]


# PARTE 2 NORMALIZACIÓN

In [8]:
array_medias = medias
array_stdev = np.sqrt(rdd5[0]/num)
print(array_stdev)
print(array_medias)

[1.60589322e+03 2.41174750e+04 1.63949454e+04 1.34264218e+05
 2.33920714e+08 4.41237795e+07 5.28172581e+00 2.10134561e+00
 9.03431222e+01 7.13843443e+08 1.30153195e+09]
[1.28153827e+03 2.12827672e+04 6.94805718e+03 6.26311961e+04
 1.22198595e+08 1.57222361e+07 9.10731393e+00 1.88975480e+00
 1.24270159e+02 2.13060301e+09 2.26149180e+09]


In [9]:
def normalize(tup):
    data = tup[0]
    normalized_data = []
    i = 0
    for e in data:
        normalized_data.append((e - array_medias[i]) / array_stdev[i])  
        i+=1
    return np.array(normalized_data), tup[1]

In [10]:
rdd2_1 = rdd1.map(normalize)
med = rdd2_1.map(lambda x: x[0]).mean()
print(med)
#print(rdd2_1.collect())
stdevv = rdd2_1.map(lambda x: x[0]).stdev()
print(stdevv)
#print(rdd1.collect())
#print(rdd2_1.collect())

[ 2.12292098e-13  1.59698094e-13 -1.56219737e-13 -3.27623399e-14
 -1.84865090e-13  3.84528564e-15  7.29247825e-13  1.20991119e-12
 -3.54308698e-13  4.06503878e-13 -1.77615382e-13]


[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


# Parte 3. Entrenamiento

In [11]:
def train(RDD_Xy, iterations, learning_rate):
    # Initialize weights (w) and bias (b)
    n_features = 11  # As stated in the problem, there are 11 features
    w = [0.0] * n_features  # Initialize weights to 0 for each feature
    b = 0.0  # Initialize bias to 0

    m = RDD_Xy.count()  # Get the number of training examples

    for i in range(iterations):
        # Compute the gradient for weights and bias
        gradients = RDD_Xy.map(lambda xy: compute_gradients(xy, w, b)).reduce(lambda g1, g2: add_gradients(g1, g2))
        
        dw = [gradients[j] / m for j in range(n_features)]  # Average the weight gradients
        db = gradients[-1] / m  # Average the bias gradient

        # Update the weights and bias using gradient descent
        w = [w[j] - learning_rate * dw[j] for j in range(n_features)]
        b = b - learning_rate * db

        # Calculate and print the cost for the current iteration
        cost = compute_cost(RDD_Xy, w, b, m)
        print(f"iteration {i}  Cost: {cost}")

    return w, b

def compute_gradients(xy, w, b):
    """
    Compute gradients for a single example (X, y).
    Returns:
    A list containing [dw1, dw2, ..., dwk, db]
    """
    X, y = xy
    z = sum([w[j] * X[j] for j in range(len(X))]) + b  # Linear combination z = w.X + b
    y_hat = 1 / (1 + math.exp(-z))  # Sigmoid function
    
    # Compute the gradients
    dz = y_hat - y
    dw = [dz * X[j] for j in range(len(X))]  # Derivative with respect to each weight
    db = dz  # Derivative with respect to the bias

    return dw + [db]

def add_gradients(g1, g2):
    """
    Add two gradient vectors element-wise.
    """
    return [g1[j] + g2[j] for j in range(len(g1))]

def compute_cost(RDD_Xy, w, b, m):
    """
    Compute the cost (log loss) for the current weights w and bias b.

    Arguments:
    RDD_Xy -- RDD containing tuples (X, y) where:
              X is the feature vector (list of floats)
              y is the actual label (0 or 1)
    w -- list of weights (length = number of features)
    b -- bias (float)
    m -- number of examples (integer)

    Returns:
    cost -- the value of the cost function (float)
    """
    cost_sum = RDD_Xy.map(lambda xy: cost_per_example(xy, w, b)).sum()
    cost = -cost_sum / m
    return cost

def cost_per_example(xy, w, b):
    """
    Compute the cost for a single example (X, y).
    """
    X, y = xy
    z = sum([w[j] * X[j] for j in range(len(X))]) + b  # Linear combination z = w.X + b
    y_hat = 1 / (1 + math.exp(-z))  # Sigmoid function

    # Avoid log(0) by adding a small value (epsilon)
    epsilon = 1e-12
    cost = y * math.log(y_hat + epsilon) + (1 - y) * math.log(1 - y_hat + epsilon)
    
    return cost

In [12]:
# Train the model
iterations = 10
learning_rate = 1.5
w, b = train(rdd2_1, iterations, learning_rate)

iteration 0  Cost: 0.3992659144762792


iteration 1  Cost: 0.329601224155895


iteration 2  Cost: 0.2935851820369425


iteration 3  Cost: 0.27064705416132845


iteration 4  Cost: 0.2544721446478405


iteration 5  Cost: 0.24234438084426718


iteration 6  Cost: 0.23286621344630484


iteration 7  Cost: 0.22523236048642015


iteration 8  Cost: 0.218940992142173


iteration 9  Cost: 0.21366083292951382


# PARTE 4 PREDICCIÓN

In [13]:
def predict(w, b, X):
    z = sum([w[j] * X[j] for j in range(len(X))]) + b  # Linear combination z = w.X + b
    y_hat = 1 / (1 + math.exp(-z))  # Sigmoid function
    
    # Convert y_hat to a binary label (0 or 1)
    return 1 if y_hat >= 0.5 else 0

def accuracy(w, b, RDD_Xy):
    # Predict for each example in RDD_Xy and compare with actual label
    correct_predictions = RDD_Xy.map(lambda xy: 1 if predict(w, b, xy[0]) == xy[1] else 0).sum()
    
    # Get the total number of examples in the RDD
    total_examples = RDD_Xy.count()
    
    # Compute accuracy as the fraction of correct predictions
    accuracy = correct_predictions / total_examples
    
    return accuracy

# PARTE 5 PRECISIÓN

In [14]:
# Calculate accuracy
acc = accuracy(w, b, rdd2_1)
print(f"Accuracy: {acc * 100:.4f}%")

Accuracy: 92.940700%
